Competition: https://www.kaggle.com/competitions/learning-agency-lab-automated-essay-scoring-2

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

2024-05-07 09:42:38.505398: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 09:42:38.505596: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 09:42:38.508604: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 09:42:38.546973: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 09:42:39.335549: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
path_data = "/home/schlemuel/OneDrive/Documentos/Bancos de dados/Kaggle/automated-essay-scoring"

# Seeing

In [33]:
df_train = pd.read_csv(path_data + '/train.csv')
df_train.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [34]:
maximum = 0

for line in range(len(df_train)):
    words = df_train.loc[2, 'full_text'].count(' ')

    if maximum < words:
        maximum = words

print(maximum)

555


In [44]:
df_test = pd.read_csv(path_data + '/test.csv')
df_test.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [36]:
df_submission = pd.read_csv(path_data + '/sample_submission.csv')
df_submission.head()

,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4


# Training

In [45]:
df_train, df_test = train_test_split(df_train, test_size=0.12, random_state=37)

In [46]:
print(df_train.shape)
print(df_test.shape)

(15230, 3)
(2077, 3)


## Many

In [20]:
%%time

max_len = maximum + 300

parameter = {
    'maxW': 1000,
    'size':  100,
    'Conv': 64,
    'NeuDen': 64,
    'lr': 0.05,
    'epoch': 1
}

tokenizer = Tokenizer(num_words=parameter['maxW'])
tokenizer.fit_on_texts(df_train['full_text'])
sequences_train = tokenizer.texts_to_sequences(df_train['full_text'])

x_train = pad_sequences(sequences_train, maxlen=max_len)
y_train = pd.get_dummies(df_train['score']).values

model = Sequential()
model.add(Embedding(parameter['maxW'], parameter['size'], input_length=max_len))
model.add(Conv1D(parameter['Conv'], 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(parameter['NeuDen'], activation='relu'))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer=Adam(learning_rate=parameter['lr']), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=parameter['epoch'])

/home/schlemuel/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


476/476 ━━━━━━━━━━━━━━━━━━━━ 21s 42ms/step - accuracy: 0.3522 - loss: 0.4218
CPU times: user 1min 12s, sys: 755 ms, total: 1min 13s
Wall time: 25.8 s


In [23]:
%%time

def test(df_test, model):
    def predict_message(text):
        sequences_test = tokenizer.texts_to_sequences(text)
        predict = pad_sequences(sequences_test, maxlen=max_len)
        prediction = model.predict(predict, verbose=-1)
        result = list(prediction[0]).index(max(prediction[0])) + 1
        return result

    predictions = np.array([predict_message(text) for text in df_test['full_text']])
    correct_predictions = (predictions == df_test_temp['score']).sum()

    return f'{round(correct_predictions / len(df_test_temp), 8) * 100}%'

df_test_temp = df_test.reset_index()
del df_test_temp['index']
print(test(df_test_temp, model))

KeyboardInterrupt: 

In [24]:
from itertools import product

max_len = maximum + 300

df_test_temp = df_test.reset_index()
del df_test_temp['index']

# Função para treinar e testar o modelo com uma determinada configuração de parâmetros
def train_and_test(df_train, df_test, parameter):
    tokenizer = Tokenizer(num_words=parameter['maxW'])
    tokenizer.fit_on_texts(df_train['full_text'])
    sequences_train = tokenizer.texts_to_sequences(df_train['full_text'])
    
    x_train = pad_sequences(sequences_train, maxlen=max_len)
    y_train = pd.get_dummies(df_train['score']).values

    model = Sequential()
    model.add(Embedding(parameter['maxW'], parameter['size'], input_length=max_len))
    model.add(Conv1D(parameter['Conv'], 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(parameter['NeuDen'], activation='relu'))
    model.add(Dense(6, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=parameter['lr']), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=0)

    sequences_test = tokenizer.texts_to_sequences(df_test['full_text'])
    predict = pad_sequences(sequences_test, maxlen=max_len)

    def predict_message(text):
        sequences_test = tokenizer.texts_to_sequences(text)
        predict = pad_sequences(sequences_test, maxlen=max_len)
        prediction = model.predict(predict, verbose=-1)
        result = list(prediction[0]).index(max(prediction[0])) + 1
        return result

    predictions = np.array([predict_message(text) for text in df_test['full_text']])
    accuracy = (predictions == df_test['score']).sum() / len(df_test)
    
    return accuracy, parameter

# Parâmetros para testar
parameters = {
    'maxW': [3000, 10000],
    'size': [100],
    'Conv': [32, 64],
    'NeuDen': [32, 64],
    'lr': [0.001, 0.01, 0.1]
}

# Melhor resultado encontrado até agora
best_accuracy = 0
best_parameter = None

# Testar todas as combinações de parâmetros
for maxW, size, Conv, NeuDen, lr in product(*parameters.values()):
    parameter = {
        'maxW': maxW,
        'size': size,
        'Conv': Conv,
        'NeuDen': NeuDen,
        'lr': lr
    }
    accuracy, config = train_and_test(df_train, df_test_temp, parameter)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_parameter = config

print(f"Melhor precisão: {best_accuracy}")
print("Configuração do melhor resultado:")
print(best_parameter)


/home/schlemuel/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Melhor precisão: 0.37217140105922003
Configuração do melhor resultado:
{'maxW': 3000, 'size': 100, 'Conv': 32, 'NeuDen': 32, 'lr': 0.1}


## Only one

In [132]:
max_words = 3000
max_len = maximum + 300

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_train['full_text'])
sequences_train = tokenizer.texts_to_sequences(df_train['full_text'])

x_train = pad_sequences(sequences_train, maxlen=max_len)
y_train = pd.get_dummies(df_train['score']).values

model = Sequential()
model.add(Embedding(max_words, 100, input_length=max_len))
model.add(Conv1D(32, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/home/schlemuel/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_42"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_42 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_42 (Conv1D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_42         │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_92 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_93 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_94 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_95 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [133]:
%%time
model.fit(x_train, y_train, batch_size=32, epochs=7)

Epoch 1/7
476/476 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.3693 - loss: 0.3923
Epoch 2/7
476/476 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.4972 - loss: 0.3184
Epoch 3/7
476/476 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.5653 - loss: 0.2914
Epoch 4/7
476/476 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - accuracy: 0.6240 - loss: 0.2625
Epoch 5/7
476/476 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.6678 - loss: 0.2379
Epoch 6/7
476/476 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.7215 - loss: 0.2107
Epoch 7/7
476/476 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.7695 - loss: 0.1841
CPU times: user 5min 3s, sys: 3.7 s, total: 5min 6s
Wall time: 1min 35s


# Predicting

In [134]:
df_test.head()

,essay_id,full_text,score
6375,5f51576,How can a computer recognize the subtle facial...,1
12813,bcd7018,to begin with people say that cars are the num...,2
9789,90b0f95,"Dear State Senator,\n\nHave you ever sent your...",2
12876,bd6f060,Being a Seagoing Cowboy is an amazing thing to...,4
4144,3d2d66c,I think that more people should join the Seago...,2


In [135]:
df_test_temp = df_test.reset_index()
del df_test_temp['index']
df_test_temp.head()

,essay_id,full_text,score
0,5f51576,How can a computer recognize the subtle facial...,1
1,bcd7018,to begin with people say that cars are the num...,2
2,90b0f95,"Dear State Senator,\n\nHave you ever sent your...",2
3,bd6f060,Being a Seagoing Cowboy is an amazing thing to...,4
4,3d2d66c,I think that more people should join the Seago...,2


In [136]:
x_test = pd.DataFrame({'essay_id': df_test_temp['essay_id'], 'full_text': df_test_temp['full_text']})
y_test = pd.DataFrame({'essay_id': df_test_temp['essay_id'], 'score': df_test_temp['score']})

In [137]:
x_test.head()

,essay_id,full_text
0,5f51576,How can a computer recognize the subtle facial...
1,bcd7018,to begin with people say that cars are the num...
2,90b0f95,"Dear State Senator,\n\nHave you ever sent your..."
3,bd6f060,Being a Seagoing Cowboy is an amazing thing to...
4,3d2d66c,I think that more people should join the Seago...


In [138]:
y_test.head()

,essay_id,score
0,5f51576,1
1,bcd7018,2
2,90b0f95,2
3,bd6f060,4
4,3d2d66c,2


In [139]:
%%time

# function to predict texts based on model
def predict_message(pred_text):
    text = pd.DataFrame({'full_text': [pred_text]})
    sequence = tokenizer.texts_to_sequences(text['full_text'])
    predict = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(predict, verbose=-1)
    result = list(prediction[0]).index(max(prediction[0])) + 1

    return result

correct = 0

for line in range(len(x_test)):
    pred_text = x_test['full_text'][line]
    prediction = predict_message(pred_text)

    if prediction == y_test['score'][line]:
    correct += 1

print(round(correct / len(x_test), 6))

0.522388
CPU times: user 1min 33s, sys: 1.56 s, total: 1min 35s
Wall time: 1min 32s
